In [1]:
from IPython.display import Image
from pathlib import Path
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import log_loss
import math


In [2]:
DATASET_PATH='/home/ivan/projects/datasets/wildlife'

DATA_PATH = Path(DATASET_PATH) 

In [3]:
train_labels = pd.read_csv(str(DATA_PATH/"train_labels.csv"), index_col="seq_id")
train_labels_columns=train_labels.columns
train_labels['season'] = train_labels.index.map(lambda x: x.split("#")[0])


In [4]:
feature_classes=list(train_labels_columns)
feature_classes.remove('reptiles')
feature_classes.remove('caracal')
feature_classes.remove('wildcat')
feature_classes.remove('honeybadger')
feature_classes.remove('vulture')
feature_classes.remove('duiker')
feature_classes.remove('civet')
feature_classes.remove('genet')
feature_classes.remove('rhinoceros')
feature_classes.remove('zorilla')
feature_classes.remove('cattle')
feature_classes.remove('steenbok')
feature_classes.remove('bat')


In [5]:
# everything_S10 = pd.read_csv("../../predictions_SER_S10.csv", index_col="seq_id_buf")
# everything_S9 = pd.read_csv("../../predictions_SER_S9.csv", index_col="seq_id_buf")

everything_S10 = pd.read_csv("../../everything_fullhd_SER_S10_all.csv", index_col="seq_id")
everything_S9 = pd.read_csv("../../everything_fullhd_SER_S9_all.csv", index_col="seq_id_buf")

everything_S10.index.name='seq_id'
everything_S9.index.name='seq_id'


In [6]:
everything = pd.concat([everything_S9,  everything_S10])


In [7]:
join = everything.join(train_labels, how='left')


In [8]:
len(join)

619405

In [9]:
len(join[join['season']=='SER_S9'])

367394

In [10]:
len(join[join['season']=='SER_S10'])

252011

In [11]:
models = [
            'pred_model_480_360_as512_img',
            'pred_model_480_360_as480_img',
            'pred_model_480_360_v2_as512_img', 
            'pred_model_480_360_v2_as480_img', 
            'pred_model_512_384_v2_as512_img'
         ]
models2 = [
           'pred_mean_back_2models_as512_',
           'pred_mean_back_2models_as480_',
          ]

models3 = [
           'pred_back_as480_',
           'pred_back_as512_',
          ]


In [12]:
interesting_columns = ['count_imgs', 'season']
models1 = []
for m in models:
    models1.append(m + '1_')
    models1.append(m + '2_')
    models1.append(m + '3_')
for m in models1 + models2 + models3 + [""]:
    for c in feature_classes:
        interesting_columns.append(m + c)
        

In [13]:
join=join[interesting_columns]

In [14]:
for c in feature_classes:
    print(c)
    join['median_img1_' + c] = join[[m + '1_' + c for m in models]].median(axis=1)
    join['median_img2_' + c]=join['median_img1_' + c]
    join['median_img3_' + c]=join['median_img1_' + c]
    join.loc[join.count_imgs > 1, 'median_img2_' + c] = join.loc[join.count_imgs > 1, [m + '2_' + c for m in models]].median(axis=1)
    join.loc[join.count_imgs > 2, 'median_img3_' + c] = join.loc[join.count_imgs > 2, [m + '3_' + c for m in models]].median(axis=1)
    join['mean_imgs_' + c] = (join['median_img1_' + c] + join['median_img2_' + c]*0.8 + join['median_img3_' + c]*0.5) /2.3
    
    join['mean_combines_' + c] = join['mean_imgs_' + c]
    join.loc[join.count_imgs > 1, 'mean_combines_' + c] = join.loc[join.count_imgs > 1,[m  + c for m in models2]].median(axis=1)
    
    join['mean_backs_' + c] = join['mean_imgs_' + c]
    join.loc[join.count_imgs > 1, 'mean_backs_' + c] = join.loc[join.count_imgs > 1,[m  + c for m in models3]].median(axis=1)
    

aardvark
aardwolf
baboon
batearedfox
buffalo
bushbuck
cheetah
dikdik
eland
elephant
empty
gazellegrants
gazellethomsons
giraffe
guineafowl
hare
hartebeest
hippopotamus
hyenaspotted
hyenastriped
impala
insectspider
jackal
koribustard
leopard
lionfemale
lionmale
mongoose
monkeyvervet
ostrich
otherbird
porcupine
reedbuck
rodents
secretarybird
serval
topi
warthog
waterbuck
wildebeest
zebra


In [15]:
train_metadata = pd.read_csv(DATA_PATH / "train_metadata.csv", parse_dates= ['datetime'], index_col="seq_id")
train_metadata = train_metadata.groupby('seq_id').first()
train_metadata['month']=train_metadata.datetime.dt.month
train_metadata['hour']=train_metadata.datetime.dt.hour
join = join.join(train_metadata.hour, how='left')
join = join.join(train_metadata.month, how='left')


In [16]:

join.loc[join.count_imgs<2, ['pred_back_as480_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_back_as512_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_mean_back_2models_as480_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_mean_back_2models_as512_' + c for c in feature_classes]]=-0.01

join.loc[join.count_imgs<3, ['pred_model_480_360_as480_img3_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_model_480_360_as480_img2_' + c for c in feature_classes]]=-0.01

join.loc[join.count_imgs<3, ['pred_model_480_360_as512_img3_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_model_480_360_as512_img2_' + c for c in feature_classes]]=-0.01

join.loc[join.count_imgs<3, ['pred_model_480_360_v2_as480_img3_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_model_480_360_v2_as480_img2_' + c for c in feature_classes]]=-0.01

join.loc[join.count_imgs<3, ['pred_model_480_360_v2_as512_img3_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_model_480_360_v2_as512_img2_' + c for c in feature_classes]]=-0.01

join.loc[join.count_imgs<3, ['pred_model_512_384_v2_as512_img3_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_model_512_384_v2_as512_img2_' + c for c in feature_classes]]=-0.01

           



           

In [17]:

test_index = set(join[join.season=='SER_S10'].index[::2])
test_dataset     = join[join.index.isin(test_index)]
val_dataset = join[(join.season=='SER_S10') & (~join.index.isin(test_index))]
train_dataset = join[join.season=='SER_S9']
animal_columns =list(train_labels.columns)
animal_columns.remove('empty')

len(train_dataset), len(val_dataset), len(test_dataset)

(367394, 126005, 126006)

In [18]:
import numpy as np
lgbt_models = ['empty', 'wildebeest', 'zebra', 'gazellethomsons', 'buffalo',
'elephant', 'hartebeest', 'impala', 'gazellegrants', 'giraffe',
'warthog', 'guineafowl', 'otherbird', 'hyenaspotted', 'lionfemale', 
'eland', 'hippopotamus', 'reedbuck', 'topi', 'baboon', 'dikdik', 
'secretarybird', 'jackal']

# training_distibution = join.loc[train_labels.season.isin(['SER_S9']), lgbt_models].mean(axis=0)
training_distibution = train_dataset[lgbt_models].mean(axis=0)

test_distibution = train_labels[lgbt_models].mean(axis=0)

scale_pos_weight=test_distibution/training_distibution

for key, value in scale_pos_weight.items():
    # do something with value
    scale_pos_weight[key] = np.clip(value,0.6,1.8)
scale_pos_weight

empty              0.967163
wildebeest         1.155221
zebra              0.936143
gazellethomsons    0.953444
buffalo            1.661107
elephant           1.265842
hartebeest         1.800000
impala             1.763873
gazellegrants      1.592588
giraffe            1.443265
warthog            1.201960
guineafowl         1.800000
otherbird          1.248398
hyenaspotted       0.902223
lionfemale         1.354255
eland              1.800000
hippopotamus       0.983885
reedbuck           1.325607
topi               1.011262
baboon             0.992658
dikdik             1.190888
secretarybird      0.810593
jackal             0.657136
dtype: float64

In [19]:
from sklearn.metrics import log_loss
loss = 0
for c in feature_classes:
    x = test_dataset[['mean_imgs_' + c, c]].values
    if sum(x[:,1]) !=0:
#         print(c)
        l = log_loss(x[:,1], x[:,0])
        loss+=l
        print(c, l)
print('simple ensembling score ', loss/54, 'Note : exclude low frequent animals')

aardvark 0.00011448686451299324
aardwolf 0.00043729597511019916
baboon 0.0008356553283192734
batearedfox 0.0004525687805471574
buffalo 0.004649831755767214
bushbuck 0.00022754214975594976
cheetah 0.0008176263029226388
dikdik 0.0012679597912138277
eland 0.001985909954805522
elephant 0.004542314817608036
empty 0.046468912787906824
gazellegrants 0.00493314393725784
gazellethomsons 0.007162891533011847
giraffe 0.0014877684568873565
guineafowl 0.0016045421577806074
hare 0.0002419734081486537
hartebeest 0.00334506707158637
hippopotamus 0.0006021332251466067
hyenaspotted 0.0016039266866717564
hyenastriped 0.00032041780880513634
impala 0.004109862442070509
insectspider 0.00506193909758066
jackal 0.0007827833027979016
koribustard 0.000676956069150987
leopard 0.00020572789264205278
lionfemale 0.0022604673769773967
lionmale 0.0012948211812802955
mongoose 0.0007801134795469051
monkeyvervet 0.00040644369353369465
ostrich 0.0007105080024520494
otherbird 0.010259745096893182
porcupine 0.0003713270289

In [20]:
import gc
del train_metadata
del everything_S10
del everything_S9
del train_labels
del join
gc.collect()

0

In [21]:


num_leaves={

}

max_depth={


}
lambda_l1={
    'empty': 0.3,
    'zebra': 0.2,
    'buffalo': 0.3,
}
lambda_l2={
    'empty': 0.3,
    'zebra': 0.2,
    'buffalo': 0.3,
}

learning_rate={
    'zebra':0.008,
    'empty': 0.008,
}

feature_fraction={
    'zebra':0.5,
}
bagging_fraction={
    'zebra':0.8,
    'empty':0.8, #?
}
bagging_freq={
    'zebra':5,
    'empty':5, #?
}
feature_fraction={
    'zebra':0.6,
    'empty':0.3,
}
# elephant
min_child_samples={
    'empty':30,
    'zebra':30,
    'buffalo':30,
    'topi':30,
    'wildebeest':30,
    'gazellethomsons':30
}

steps={
    'empty' : int(907*1.1),
    'wildebeest' : int(865*1.1),
    'zebra' : int(1050*1.1),
    'gazellethomsons' : int(1072*1.1),
    'buffalo' : int(589*1.1),
    'elephant' : int(563*1.1),
    'hartebeest' : int(620*1.1),
    'impala' : int(630*1.1),
    'gazellegrants' : int(616*1.1),
    'giraffe' : int(705*1.1),
    'warthog' : int(752*1.1),
    'guineafowl' : int(416*1.1),
    'otherbird' : int(348*1.1),


}
clazz='wildebeest'

columns = ['pred_model_480_360_as480_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_as480_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_as480_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img3_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img3_' + c for c in feature_classes] +\
    ['mean_backs_' + c for c in feature_classes] +\
    ['median_img1_' + c for c in feature_classes] +\
    ['median_img2_' + c for c in feature_classes] +\
    ['median_img3_' + c for c in feature_classes] +\
    ['mean_combines_' + c for c in feature_classes] +\
    ['hour', 'month']



In [22]:
X_train = train_dataset[columns]
y_train = train_dataset[clazz]
X_val = val_dataset[columns]
y_val = val_dataset[clazz]

In [23]:

clazzs = ['empty', 'wildebeest', 'zebra', 'gazellethomsons', 'buffalo',
'elephant', 'hartebeest', 'impala', 'gazellegrants', 'giraffe',
'warthog', 'guineafowl', 'otherbird']


        
for clazz in clazzs:
    
#     feature_classes=feature_classes_d.get(clazz, list(train_labels_columns))

    columns = ['pred_model_480_360_as480_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_as480_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_as480_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img3_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img3_' + c for c in feature_classes] +\
    ['mean_backs_' + c for c in feature_classes] +\
    ['median_img1_' + c for c in feature_classes] +\
    ['median_img2_' + c for c in feature_classes] +\
    ['median_img3_' + c for c in feature_classes] +\
    ['mean_combines_' + c for c in feature_classes] +\
    ['hour', 'month']



    X_train = train_dataset[columns]
    y_train = train_dataset[clazz]
    X_val = val_dataset[columns]
    y_val = val_dataset[clazz]

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

    params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': {'binary_logloss'},
                'num_leaves': num_leaves.get(clazz, 32),
                'max_depth': max_depth.get(clazz, 6),
                'learning_rate': learning_rate.get(clazz, 0.009),
                'feature_fraction':  feature_fraction.get(clazz, 0.6),
                'bagging_fraction': bagging_fraction.get(clazz, 0.8),
                'bagging_freq': bagging_freq.get(clazz, 5),
                'scale_pos_weight':scale_pos_weight.get(clazz, 1),
                'lambda_l1':lambda_l1.get(clazz, 0.2),
                'lambda_l2':lambda_l2.get(clazz, 0.2),
                'min_child_samples':min_child_samples.get(clazz, 25),

    }

    gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=3000,
                        valid_sets=[lgb_train, lgb_val],
                        early_stopping_rounds=20)
    gbm.save_model(clazz + '.txt', num_iteration=gbm.best_iteration) 

    X_test = test_dataset[columns]
    y_test = test_dataset[clazz]
    
    print(clazz, log_loss(y_test, gbm.predict(X_test.values)))
    gc.collect()


[1]	training's binary_logloss: 0.512129	valid_1's binary_logloss: 0.511815
Training until validation scores don't improve for 20 rounds
[2]	training's binary_logloss: 0.504831	valid_1's binary_logloss: 0.504511
[3]	training's binary_logloss: 0.497759	valid_1's binary_logloss: 0.497426
[4]	training's binary_logloss: 0.49088	valid_1's binary_logloss: 0.490537
[5]	training's binary_logloss: 0.484208	valid_1's binary_logloss: 0.483849
[6]	training's binary_logloss: 0.477714	valid_1's binary_logloss: 0.477346
[7]	training's binary_logloss: 0.471395	valid_1's binary_logloss: 0.471012
[8]	training's binary_logloss: 0.465245	valid_1's binary_logloss: 0.464863
[9]	training's binary_logloss: 0.459267	valid_1's binary_logloss: 0.458864
[10]	training's binary_logloss: 0.453422	valid_1's binary_logloss: 0.453015
[11]	training's binary_logloss: 0.447745	valid_1's binary_logloss: 0.447321
[12]	training's binary_logloss: 0.44217	valid_1's binary_logloss: 0.441749
[13]	training's binary_logloss: 0.4367

[109]	training's binary_logloss: 0.177683	valid_1's binary_logloss: 0.178136
[110]	training's binary_logloss: 0.176308	valid_1's binary_logloss: 0.176772
[111]	training's binary_logloss: 0.174942	valid_1's binary_logloss: 0.175426
[112]	training's binary_logloss: 0.173596	valid_1's binary_logloss: 0.17409
[113]	training's binary_logloss: 0.172271	valid_1's binary_logloss: 0.172776
[114]	training's binary_logloss: 0.170998	valid_1's binary_logloss: 0.171514
[115]	training's binary_logloss: 0.169691	valid_1's binary_logloss: 0.170217
[116]	training's binary_logloss: 0.168404	valid_1's binary_logloss: 0.168947
[117]	training's binary_logloss: 0.167125	valid_1's binary_logloss: 0.167679
[118]	training's binary_logloss: 0.165859	valid_1's binary_logloss: 0.166425
[119]	training's binary_logloss: 0.164609	valid_1's binary_logloss: 0.165181
[120]	training's binary_logloss: 0.163372	valid_1's binary_logloss: 0.163953
[121]	training's binary_logloss: 0.162144	valid_1's binary_logloss: 0.162738


[216]	training's binary_logloss: 0.0868015	valid_1's binary_logloss: 0.0883548
[217]	training's binary_logloss: 0.0863114	valid_1's binary_logloss: 0.0878705
[218]	training's binary_logloss: 0.0858253	valid_1's binary_logloss: 0.0873902
[219]	training's binary_logloss: 0.0853422	valid_1's binary_logloss: 0.0869114
[220]	training's binary_logloss: 0.0848617	valid_1's binary_logloss: 0.0864381
[221]	training's binary_logloss: 0.0843913	valid_1's binary_logloss: 0.0859775
[222]	training's binary_logloss: 0.0839177	valid_1's binary_logloss: 0.0855139
[223]	training's binary_logloss: 0.0834465	valid_1's binary_logloss: 0.0850556
[224]	training's binary_logloss: 0.082987	valid_1's binary_logloss: 0.0846062
[225]	training's binary_logloss: 0.0825262	valid_1's binary_logloss: 0.0841553
[226]	training's binary_logloss: 0.0820653	valid_1's binary_logloss: 0.0837082
[227]	training's binary_logloss: 0.0816108	valid_1's binary_logloss: 0.0832628
[228]	training's binary_logloss: 0.0811645	valid_1's 

[321]	training's binary_logloss: 0.0525723	valid_1's binary_logloss: 0.0551004
[322]	training's binary_logloss: 0.0523792	valid_1's binary_logloss: 0.0549131
[323]	training's binary_logloss: 0.0521781	valid_1's binary_logloss: 0.0547195
[324]	training's binary_logloss: 0.0519768	valid_1's binary_logloss: 0.054528
[325]	training's binary_logloss: 0.0517763	valid_1's binary_logloss: 0.0543371
[326]	training's binary_logloss: 0.0515784	valid_1's binary_logloss: 0.0541528
[327]	training's binary_logloss: 0.0513831	valid_1's binary_logloss: 0.0539653
[328]	training's binary_logloss: 0.0511916	valid_1's binary_logloss: 0.053784
[329]	training's binary_logloss: 0.0509993	valid_1's binary_logloss: 0.053603
[330]	training's binary_logloss: 0.0508092	valid_1's binary_logloss: 0.053419
[331]	training's binary_logloss: 0.0506214	valid_1's binary_logloss: 0.0532406
[332]	training's binary_logloss: 0.0504354	valid_1's binary_logloss: 0.0530674
[333]	training's binary_logloss: 0.0502483	valid_1's bin

[425]	training's binary_logloss: 0.0382062	valid_1's binary_logloss: 0.0417219
[426]	training's binary_logloss: 0.0381168	valid_1's binary_logloss: 0.041645
[427]	training's binary_logloss: 0.0380287	valid_1's binary_logloss: 0.0415654
[428]	training's binary_logloss: 0.037941	valid_1's binary_logloss: 0.0414874
[429]	training's binary_logloss: 0.0378551	valid_1's binary_logloss: 0.0414072
[430]	training's binary_logloss: 0.037768	valid_1's binary_logloss: 0.0413242
[431]	training's binary_logloss: 0.0376811	valid_1's binary_logloss: 0.0412459
[432]	training's binary_logloss: 0.0375961	valid_1's binary_logloss: 0.0411678
[433]	training's binary_logloss: 0.0375127	valid_1's binary_logloss: 0.04109
[434]	training's binary_logloss: 0.0374292	valid_1's binary_logloss: 0.0410174
[435]	training's binary_logloss: 0.0373459	valid_1's binary_logloss: 0.0409447
[436]	training's binary_logloss: 0.0372627	valid_1's binary_logloss: 0.0408648
[437]	training's binary_logloss: 0.0371809	valid_1's bina

...

[39]	training's binary_logloss: 0.0111496	valid_1's binary_logloss: 0.0132444
[40]	training's binary_logloss: 0.0110698	valid_1's binary_logloss: 0.0131803
[41]	training's binary_logloss: 0.0109927	valid_1's binary_logloss: 0.0131163
[42]	training's binary_logloss: 0.0109136	valid_1's binary_logloss: 0.0130527
[43]	training's binary_logloss: 0.0108377	valid_1's binary_logloss: 0.0129913
[44]	training's binary_logloss: 0.0107616	valid_1's binary_logloss: 0.0129353
[45]	training's binary_logloss: 0.01069	valid_1's binary_logloss: 0.0128783
[46]	training's binary_logloss: 0.0106184	valid_1's binary_logloss: 0.0128248
[47]	training's binary_logloss: 0.0105491	valid_1's binary_logloss: 0.0127707
[48]	training's binary_logloss: 0.0104827	valid_1's binary_logloss: 0.0127184
[49]	training's binary_logloss: 0.0104165	valid_1's binary_logloss: 0.0126621
[50]	training's binary_logloss: 0.0103534	valid_1's binary_logloss: 0.012609
[51]	training's binary_logloss: 0.0102901	valid_1's binary_logloss:

[143]	training's binary_logloss: 0.00713643	valid_1's binary_logloss: 0.0104285
[144]	training's binary_logloss: 0.00711646	valid_1's binary_logloss: 0.0104181
[145]	training's binary_logloss: 0.00709488	valid_1's binary_logloss: 0.0104091
[146]	training's binary_logloss: 0.00707683	valid_1's binary_logloss: 0.0104
[147]	training's binary_logloss: 0.00705843	valid_1's binary_logloss: 0.0103889
[148]	training's binary_logloss: 0.0070401	valid_1's binary_logloss: 0.0103783
[149]	training's binary_logloss: 0.00702176	valid_1's binary_logloss: 0.0103703
[150]	training's binary_logloss: 0.00700349	valid_1's binary_logloss: 0.0103601
[151]	training's binary_logloss: 0.00698448	valid_1's binary_logloss: 0.010352
[152]	training's binary_logloss: 0.00696817	valid_1's binary_logloss: 0.0103446
[153]	training's binary_logloss: 0.0069516	valid_1's binary_logloss: 0.0103359
[154]	training's binary_logloss: 0.00693511	valid_1's binary_logloss: 0.0103279
[155]	training's binary_logloss: 0.00691847	va

[246]	training's binary_logloss: 0.00577834	valid_1's binary_logloss: 0.00985707
[247]	training's binary_logloss: 0.00576973	valid_1's binary_logloss: 0.00985585
[248]	training's binary_logloss: 0.00575964	valid_1's binary_logloss: 0.00985436
[249]	training's binary_logloss: 0.00574991	valid_1's binary_logloss: 0.00985253
[250]	training's binary_logloss: 0.00573981	valid_1's binary_logloss: 0.00985038
[251]	training's binary_logloss: 0.00573106	valid_1's binary_logloss: 0.00984784
[252]	training's binary_logloss: 0.00572095	valid_1's binary_logloss: 0.0098473
[253]	training's binary_logloss: 0.00571315	valid_1's binary_logloss: 0.00984446
[254]	training's binary_logloss: 0.00570521	valid_1's binary_logloss: 0.00984283
[255]	training's binary_logloss: 0.00569673	valid_1's binary_logloss: 0.00983855
[256]	training's binary_logloss: 0.00568771	valid_1's binary_logloss: 0.00983662
[257]	training's binary_logloss: 0.00567774	valid_1's binary_logloss: 0.00983347
[258]	training's binary_loglo

[348]	training's binary_logloss: 0.00504441	valid_1's binary_logloss: 0.0097235
[349]	training's binary_logloss: 0.00503808	valid_1's binary_logloss: 0.00972241
[350]	training's binary_logloss: 0.00503149	valid_1's binary_logloss: 0.00972034
[351]	training's binary_logloss: 0.00502619	valid_1's binary_logloss: 0.00971958
[352]	training's binary_logloss: 0.00501878	valid_1's binary_logloss: 0.00971986
[353]	training's binary_logloss: 0.00501313	valid_1's binary_logloss: 0.00971961
[354]	training's binary_logloss: 0.00500825	valid_1's binary_logloss: 0.00971868
[355]	training's binary_logloss: 0.00500294	valid_1's binary_logloss: 0.0097186
[356]	training's binary_logloss: 0.00499781	valid_1's binary_logloss: 0.00971916
[357]	training's binary_logloss: 0.00499295	valid_1's binary_logloss: 0.00971879
[358]	training's binary_logloss: 0.00498743	valid_1's binary_logloss: 0.00971802
[359]	training's binary_logloss: 0.00498305	valid_1's binary_logloss: 0.00971749
[360]	training's binary_loglos

[450]	training's binary_logloss: 0.00453593	valid_1's binary_logloss: 0.00969368
[451]	training's binary_logloss: 0.00453056	valid_1's binary_logloss: 0.00969349
[452]	training's binary_logloss: 0.00452518	valid_1's binary_logloss: 0.00969323
[453]	training's binary_logloss: 0.00452003	valid_1's binary_logloss: 0.00969249
[454]	training's binary_logloss: 0.00451488	valid_1's binary_logloss: 0.00969136
[455]	training's binary_logloss: 0.00450927	valid_1's binary_logloss: 0.00969072
[456]	training's binary_logloss: 0.00450505	valid_1's binary_logloss: 0.00969127
[457]	training's binary_logloss: 0.00450261	valid_1's binary_logloss: 0.00969098
[458]	training's binary_logloss: 0.00449778	valid_1's binary_logloss: 0.00968977
[459]	training's binary_logloss: 0.00449559	valid_1's binary_logloss: 0.00968977
[460]	training's binary_logloss: 0.00449307	valid_1's binary_logloss: 0.00969021
[461]	training's binary_logloss: 0.00448771	valid_1's binary_logloss: 0.00968991
[462]	training's binary_logl

In [25]:
losses = []
for clazz in clazzs:
    
#     feature_classes=feature_classes_d.get(clazz, list(train_labels_columns))

    columns = ['pred_model_480_360_as480_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_as480_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_as480_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img3_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img3_' + c for c in feature_classes] +\
    ['mean_backs_' + c for c in feature_classes] +\
    ['median_img1_' + c for c in feature_classes] +\
    ['median_img2_' + c for c in feature_classes] +\
    ['median_img3_' + c for c in feature_classes] +\
    ['mean_combines_' + c for c in feature_classes] +\
    ['hour', 'month']




    gbm = lgb.Booster(model_file=clazz + '.txt')


    X_test = test_dataset[columns]
    y_test = test_dataset[clazz]
    l = log_loss(y_test, gbm.predict(X_test.values))
    losses.append(l)
    print(clazz, l)
    gc.collect()


empty 0.032561336883759374
wildebeest 0.02134952140600164
zebra 0.010410163679976766
gazellethomsons 0.006370004320359023
buffalo 0.0045945335607014
elephant 0.0037806150024401186
hartebeest 0.003238209560658141
impala 0.004122439429679086
gazellegrants 0.004874318450402127
giraffe 0.001319812619547732
warthog 0.002547203587293033
guineafowl 0.0021142116858322057
otherbird 0.008260333987108577
new score 0.008118669551827632
